<h1><center>CS598 Deep Learning for Healthcare Spring 2023<br>Paper Reproduction Project</center></h1>

<h3><center>Gilberto Ramirez and Jay Kawkani<br><span style="font-family:monospace;">{ger6, kakwani2}@illinois.edu</span><br><font color="lightgrey">Group ID: 27 | Paper ID: 181</font></center></h3>

In this project, we aim to reproduce the paper [*Learning Task for Multitask Learning: Heterogeneous Patient Populations in the ICU* by (Suresh et al, 2018)](https://arxiv.org/abs/1806.02878). In this paper, the authors propose a novel two-step pipeline to predict in-hospital mortality across patient populations with different characteristics. The first step of the pipeline divides patients into relevant non-overlapping cohorts in an unsupervised way using a long short-term memory (LSTM) autoencoder followed by a Gaussian Mixture Model (GMM). The second step of the pipeline predicts in-hospital mortality for each patient cohort identified in the previous step using an LSTM based multi-task learning model where every cohort is considered a different task.
The paper claims that by applying this pipeline, the multi-task learning model can leverage shared knowledge across the distinct patient groups identified and it can work effectively since the groups were obtained using a data-driven method rather than relying in domain knowledge or auxiliary labels.

## Table of Contents

1. [Data](#section-1)
2. [Methods](#section-2)

## <a class="anchor" id="section-1">1. Data</a>

This paper uses the publicly available [MIMIC-III database](https://www.nature.com/articles/sdata201635) which contains clinical data in a critical care setting. After reviewing the paper in detail, we decided to use [MIMIC-Extract](https://arxiv.org/abs/1907.08322), an open source pipeline by (Wang et al., 2020) for transforming the raw EHR data into usable Pandas dataframes containing hourly time series of vitals and laboratory measurements after performing unit conversion, outlier handling, and aggregation of semantically similar features.

Unfortunately, the MIMIC-Extract pipeline misses two features the [paper code](https://github.com/mit-caml/multitask-patients) makes use of:
* `timecmo_chart` which indicates the timestamp of a patient when it has been declared in CMO (Comfort Measures Only) state. This feature comes from a MIMIC-III concept table called `code_status`.
* `sapsii` which contains the SAPS (Simplified Acute Physiology Score) II. This feature comes from another MIMIC-III concept table called `sapsii`.

As a result, there are three data files needed to run this notebook:
* `all_hourly_data.h5`, an HDF file resulting from running the MIMIC-Extract pipeline which is publicly available in GCP using [this link](https://console.cloud.google.com/storage/browser/mimic_extract) and referenced in the [MIMIC-Extract github repo](https://github.com/MLforHealth/MIMIC_Extract).
* `code_status.csv`, a CSV file holding the MIMIC concept table `CODE_STATUS` that can be generated following the instructions in [this link within the MIT-LCP github repo](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/concepts#generating-the-concepts-in-postgresql).
* `sapsii.csv`, a CSV file holding the MIMIC concept table `SAPSII` that can be generated following the instructions in [this link within the MIT-LCP github repo](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/concepts#generating-the-concepts-in-postgresql).

The functions used in this notebook assume the three files are in the folder `../data/` by default. However, location can be defined using arguments to the functions that process the data.

All code needed to replicate the paper is in [our github repo](https://github.com/ger6-illini/dl4h-sp23-team27-project) inside a Python module called `mtl_patients`.

The first function from that module we will start using is `get_summaries()`. This function provides three summaries in four dataframes which, in return order, are:
* A summary providing some statistics of all patients broken by careunit.
* A summary providing some statistics of all patients broken by SAPS-II score quartile.
* A summary providing some statistics of the 29 distinct physiological measurements used in the paper.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import sys
pathname = "../code/"
if pathname not in sys.path:
    sys.path.append("../code/")

from mtl_patients import get_summaries

2023-03-25 10:15:09.692630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%%time
pat_summ_by_cu_df, pat_summ_by_sapsiiq_df, vitals_labs_summ_df = get_summaries()

CPU times: user 6.97 s, sys: 1.43 s, total: 8.4 s
Wall time: 8.71 s


Let's now display the summaries one at a time.

### 1.1. Data summary by patients in each intensive care unit (ICU)

In [3]:
pat_summ_by_cu_df

,N,n,Class Imbalance,Age (Mean),Gender (Male)
Careunit,,,,,
CCU,5193,790,0.152,83.31,0.58
CSRU,7050,223,0.032,69.54,0.67
MICU,12207,2674,0.219,78.21,0.51
SICU,5520,829,0.150,73.49,0.51
TSICU,4502,583,0.129,67.33,0.61
Overall,34472,5099,0.148,75.03,0.57


In the previous summary, patients were broken in groups where each group is one of the five careunits where patients were first admitted:
* CCU: Coronary Care Unit
* CSRU: Cardiac Surgery Recovery Unit
* MICU: Medical Intensive Care Unit
* SICU: Surgical Intensive Care Unit
* TSICU: Trauma Surgical Intensive Care Unit

In addition, an overall group was also added. The statistics provided by the summary are:
* `N`: The number of samples (patients) in the group.
* `n`: The number of samples (patients) where meeting the in-hospital mortality criteria defined in the paper: patient died or had a note of "Do Not Resuscitate" (DNR) or had a note of "Comfort Measures Only" (CMO).
* `Class Imbalance`: Ratio of patients meeting the in-hospital mortality criteria defined in the paper, i.e., $\dfrac{\text{N}}{\text{n}}$.
* `Age (Mean)`: Mean age of patients for each group in years.
* `Gender (Male)`: Ratio of patients that are males.

This summary was prepared to match the Table 1 in the original paper. There are differences between both that can be attributed to the way how data was preprocessed by MIMIC-Extract when compared to the preprocessing done by the authors back in 2018, before MIMIC-Extract became available.

### 1.2. Data summary by patients in each SAPS-II score quartile

In [4]:
pat_summ_by_sapsiiq_df

,N,n,Class Imbalance,Age (Mean),Gender (Male),SAPS-II (Min),SAPS-II (Mean),SAPS-II (Max)
SAPS-II Quartile,,,,,,,,
0,7449,115,0.015,45.50,0.61,0,16.56,22
1,10322,669,0.065,68.84,0.58,23,27.73,32
2,8360,1274,0.152,86.70,0.55,33,36.72,41
3,8341,3041,0.365,97.36,0.53,42,52.62,118
Overall,34472,5099,0.148,75.03,0.57,0,33.52,118


In the previous summary, patients were broken based on the quartile of the SAPS-II score assigned to them. As it can be seen, the two quartiles have the ranges $[0, 22], [23, 32], [33, 41], [42, 118] $. This was included in the authors code but not in the paper. It seems the class imbalance might have been the primary reason. As it is evident from the summary, most of the patients are in quartile $3$ since they are in an ICU and is expected their values are on the high side.

### 1.3. Data summary for physiological measurements

In [5]:
vitals_labs_summ_df

,min,avg,max,std,N,pres.
Vital/Lab Measurement,,,,,,
anion gap,5.00,13.72,50.00,3.99,183732,0.0835
bicarbonate,0.00,24.23,53.00,4.74,192632,0.0875
blood urea nitrogen,0.00,26.21,250.00,21.75,194596,0.0884
chloride,50.00,105.22,175.00,6.31,211525,0.0961
creatinine,0.10,1.39,46.60,1.48,195429,0.0888
diastolic blood pressure,0.00,60.89,307.00,14.13,1908674,0.8672
fraction inspired oxygen,0.21,0.53,1.00,0.19,98315,0.0447
glascow coma scale total,3.00,12.49,15.00,3.59,377787,0.1716
glucose,33.00,140.49,1591.00,57.22,512585,0.2329


In the previous summary, all vitals and lab measurements selected in the paper (29 in total) are listed with relevant statistics associated to it:
* `min` representing the minimum of the measurement observed in the vitals/labs.
* `avg` representing the average of the measurement observed in the vitals/labs.
* `max` representing the maximum of the measurement observed in the vitals/labs.
* `std` representing the standard deviation of the measurement observed in the vitals/labs.
* `N` representing the number of non `NaN` samples for the specific vital/lab measurement.
* `pres.` representing the portion of all possible hours across all patients, admissions, and ICU stays where at least one of the 104 vitals/labs measurements in the original MIMIC-Extract pipeline was taken.

All these measurements are based on the `vitals_labs_mean` dataframe in the MIMIC-Extract pipeline which provides average of vitals/labs on a per hour basis for each patient after going into an ICU.

## <a class="anchor" id="section-2">2. Methods & Results</a>

The paper uses a two-step pipeline to: 1) identify relevant patient cohorts, and 2) use those relevant cohorts as separate tasks in a multi-lask learning framework to predict in-hospital mortality.

### 2.1. Identifying Relevant Patient Cohorts

In this first step, the raw patient data needs to be prepared in such a way that the result is a 3D matrix of shape $(P \times T \times F)$ where $P$ represents the number of patients, $T$ the number of timesteps, and $F$ the number of features as depicted in the figure below (in blue) which is partially based on Figure 2 of the original paper. All numbers shown in the figure below correspond to a specific experiment published in the paper in which the observation window is limited to the first $24$ hours (cutoff periord) after a patient goes into a careunit and there is a gap of $12$ hours (gap period) between the end of the observation window and the beginning of the prediction window where the prediction task is in-hospital mortality.

Preparation of the data to get the 3D (blue) matrix is performed by a function called `prepare_data()` inside the `mtl_patients.py` module. This preparation consists of the following transformations taken from the paper and the reference implementation:

1. Calculation of the mortality flag (prediction label) and mortality time for every patient in the dataset using an *extended* definition of mortality: death, a note of "Do Not Resuscitate" (DNR), or a note of "Comfort Measures Only" (CMO). In case any of these conditions are met for a patient, the corresponding mortality label is set to *True* and the corresponding mortality time is considered as the earliest time of any of the three conditions.
2. Data used for the prediction is only limited to the first certain amount of hours after a patient goes into the ICU. This amount of hours is called inside the code "a cutoff period" (observation window) and defines the period of data used to train all models. In addition, there is another number of hours called inside the code "the gap period" which represents the time between the end of the observation window and the beginning of the prediction window to prevent label leakage. All patients that died under the *extended* definition before the cutoff period plus the gap period or stayed less than the cutoff period are excluded from the experiment as part of this step. Also, all patients under the age of 15 are excluded (this is already part of the exclusion criteria of the MIMIC-Extract pipeline).
3. There are 29 vitals/labs timeseries selected by the paper. Only data within the cutoff period for vitals/labs is kept and rest is removed. This will be used for the rest of the machine learning pipeline.
4. All vitals/labs values are converted to z-scores so they all have zero mean and unit standard deviation. Those z-scores are rounded to the closest integer and clipped between $-4$ and $4$ or set to $9$ in case of `NaN`. This allows to map every vital/lab measurement (a float) to one of ten possible values $[-4, -3, -2, -1, 0, 1, 2, 3, 4, 9]$, so they can be converted to dummy values. After dummifying the vitals/labs, column for the $9$ (`NaN`) is removed, and the resulting matrix is sparse and containing either $0$s or $1$s.
5. Every patient is padded with rows of zeroes for those hours that are missed. For example, if a patient only has vitals/labs for the first ten hours and the cutoff period is 24, code adds fourteen hours (rows) with zeroes for that patient. In the end, the matrix will have a size of $P \times T \times F$ as expected by the subsequent models.
6. Finally, static data (gender, age, and ethnicity) is converted to integers representing categories and dummified. In case of age, there are four buckets established; $(10, 30), (30, 50), (50, 70), (70, \infty)$; while ethnicity is broken into five buckets (asian, white, hispanic, black, other).
7. Cohort assignments based on first careunit or Simplified Acute Physiology Score (SAPS) II score quartile is calculated for each patient and returned as well.

![Figure 1](../img/paper-181-fig-1.png)

The `discover_cohorts()` function inside the `mtl_patients.py` module is the one implementing the pipeline shown in the figure above and the calling the `prepare_data()` function detailed previously as the first step. Once data has been processed, the function will break the data in training, validation, and test data sets in a $70\%/10\%/20\%$ proportion.

The training data is used to train an LSTM autoencoder. The main purpose of the LSTM autoencoder is to generate a fixed-length dense representation (embedding) of the sparse inputs trying. The paper selected embeddings of size $50$ as the optimal dimension (hyperparameter). The purple box in the middle of the diagram above (a 2D matrix) represents the embeddings after the LSTM autoencoder learned the representation of the original 3D matrix of shape $(32537 \times 24 \times 232)$ where every row corresponds to a patient.

Once the embeddings are calculated, a Gaussian Mixture Model is applied using $3$ clusters (the value the authors considered optimal). The result are the three green boxes representing three cohorts discovered in an unsupervised way and grouping similar patients based on the three static and the 29 time-varying vitals/labs selected from the MIMIC-III database.

Below, the line of code calling the `discover_cohorts()` function and returning the NumPy array with cohort membership, consisting of three groups, for each patient.

In [6]:
from mtl_patients import discover_cohorts

In [7]:
%%time
cohort_unsupervised = discover_cohorts(cutoff_hours=24, gap_hours=12)

2023-03-25 10:16:10.066969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training LSTM autoencoder started...
Epoch 1/100
178/178 [==============================] - 14s 70ms/step - loss: 0.0405 - val_loss: 0.0360
Epoch 2/100
178/178 [==============================] - 12s 65ms/step - loss: 0.0347 - val_loss: 0.0333
Epoch 3/100
178/178 [==============================] - 12s 65ms/step - loss: 0.0324 - val_loss: 0.0318
Epoch 4/100
178/178 [==============================] - 12s 66ms/step - loss: 0.0312 - val_loss: 0.0308
Epoch 5/100
178/178 [==============================] - 12s 66ms/step - loss: 0.0303 - val_loss: 0.0299
Epoch 6/100
178/178 [==============================] - 12s 67ms/step - loss: 0.0295 - val_loss: 0.0293
Epoch 7/100
178/178 [==============================] - 12s 65ms/step - loss: 0.0291 - val_loss: 0.0289
Epoch 8/100
178/178 [==============================] - 11s 64ms/step - loss: 0.0287 - val_loss: 0.0286
Epoch 9/100
178/178 [==============================] - 12s 65ms/step - loss: 0.0284 - val_loss: 0.0284
Epoch 10/100
178/178 [==============

178/178 [==============================] - 13s 71ms/step - loss: 0.0236 - val_loss: 0.0238
Epoch 80/100
178/178 [==============================] - 11s 64ms/step - loss: 0.0236 - val_loss: 0.0237
Epoch 81/100
178/178 [==============================] - 13s 75ms/step - loss: 0.0236 - val_loss: 0.0237
Epoch 82/100
178/178 [==============================] - 13s 74ms/step - loss: 0.0236 - val_loss: 0.0237
Epoch 83/100
178/178 [==============================] - 13s 74ms/step - loss: 0.0235 - val_loss: 0.0237
Epoch 84/100
178/178 [==============================] - 12s 68ms/step - loss: 0.0235 - val_loss: 0.0236
Epoch 85/100
178/178 [==============================] - 11s 63ms/step - loss: 0.0235 - val_loss: 0.0236
Epoch 86/100
178/178 [==============================] - 11s 64ms/step - loss: 0.0235 - val_loss: 0.0236
Epoch 87/100
178/178 [==============================] - 12s 68ms/step - loss: 0.0235 - val_loss: 0.0236
Epoch 88/100
178/178 [==============================] - 12s 65ms/step - loss:

In [8]:
from mtl_patients import run_mortality_prediction_task, prepare_data, create_single_task_learning_model
from mtl_patients import stratified_split

In [9]:
X, Y, cohort_careunits, cohort_sapsii_quartile, subject_ids = prepare_data()

In [10]:
model_type='global'
cutoff_hours=24
gap_hours=12
cohort_criteria_to_select='careunits'
train_val_random_seed=0
cohort_unsupervised_filename='../data/unsupervised_clusters.npy'
lstm_layer_size=16
epochs=100
learning_rate=0.0001
use_cohort_inv_freq_weights=False

In [11]:
if cohort_criteria_to_select == 'careunits':
    cohort_criteria = cohort_careunits
elif cohort_criteria_to_select == 'sapsii_quartile':
    cohort_criteria = cohort_sapsii_quartile
elif cohort_criteria_to_select == 'unsupervised':
    cohort_criteria = np.load(f"{cohort_unsupervised_filename}")

In [12]:
X_train, X_val, X_test, y_train, y_val, y_test, cohorts_train, cohorts_val, cohorts_test = \
    stratified_split(X, Y, cohort_criteria, train_val_random_seed=train_val_random_seed)

In [13]:
tasks = np.unique(cohorts_train)

In [14]:
tasks

array(['CCU', 'CSRU', 'MICU', 'SICU', 'TSICU'], dtype=object)

In [15]:
task_weights = {}    
for cohort in tasks:
    num_samples_in_cohort = len(np.where(cohorts_train == cohort)[0])
    print(f"# of patients in cohort {cohort} is {str(num_samples_in_cohort)}")
    task_weights[cohort] = len(X_train) / num_samples_in_cohort

# of patients in cohort CCU is 3362
# of patients in cohort CSRU is 4858
# of patients in cohort MICU is 7976
# of patients in cohort SICU is 3633
# of patients in cohort TSICU is 2946


In [16]:
sample_weight = None

In [17]:
model = create_single_task_learning_model(lstm_layer_size=lstm_layer_size, input_dims=X_train.shape[1:],
                                          output_dims=1, learning_rate=learning_rate)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 16)                15936     
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 15,953
Trainable params: 15,953
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

In [19]:
X_train.shape

(22775, 24, 232)

In [20]:
y_train.shape

(22775,)

In [21]:
model.fit(X_train, y_train, epochs=epochs, batch_size=100)

Epoch 1/100
228/228 [==============================] - 2s 7ms/step - loss: 0.4781 - accuracy: 0.8537
Epoch 2/100
228/228 [==============================] - 1s 6ms/step - loss: 0.4028 - accuracy: 0.8692
Epoch 3/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3979 - accuracy: 0.8692
Epoch 4/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3951 - accuracy: 0.8692
Epoch 5/100
228/228 [==============================] - 2s 7ms/step - loss: 0.3935 - accuracy: 0.8692
Epoch 6/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3919 - accuracy: 0.8692
Epoch 7/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3908 - accuracy: 0.8692
Epoch 8/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3898 - accuracy: 0.8692
Epoch 9/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3890 - accuracy: 0.8692
Epoch 10/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3882 - accura

228/228 [==============================] - 1s 7ms/step - loss: 0.3518 - accuracy: 0.8715
Epoch 82/100
228/228 [==============================] - 2s 7ms/step - loss: 0.3515 - accuracy: 0.8716
Epoch 83/100
228/228 [==============================] - 1s 7ms/step - loss: 0.3505 - accuracy: 0.8717
Epoch 84/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3499 - accuracy: 0.8721
Epoch 85/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3496 - accuracy: 0.8719
Epoch 86/100
228/228 [==============================] - 1s 7ms/step - loss: 0.3489 - accuracy: 0.8721
Epoch 87/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3481 - accuracy: 0.8719
Epoch 88/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3476 - accuracy: 0.8721
Epoch 89/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3473 - accuracy: 0.8725
Epoch 90/100
228/228 [==============================] - 1s 6ms/step - loss: 0.3466 - accuracy: 

In [22]:
y_pred = model.predict(X_test)

204/204 [==============================] - 0s 2ms/step


In [23]:
df_metrics = pd.DataFrame(index=np.append(tasks, ['Macro', 'Micro']))

In [24]:
from sklearn.metrics import roc_auc_score

# calculate AUC for every cohort
lst_of_auc = []
for task in tasks:
    auc = roc_auc_score(y_test[cohorts_test == task], y_pred[cohorts_test == task])
    lst_of_auc.append(auc)
    df_metrics.loc[task, 'AUC'] = auc

# calculate macro AUC
df_metrics.loc['Macro', 'AUC'] = np.nanmean(np.array(lst_of_auc), axis=0)

# calculate micro AUC
df_metrics.loc['Micro', 'AUC'] = roc_auc_score(y_test, y_pred)

In [25]:
df_metrics

,AUC
CCU,0.509169
CSRU,0.517299
MICU,0.521928
SICU,0.489976
TSICU,0.451812
Macro,0.498037
Micro,0.506323


In [26]:
roc_auc_score(y_test, y_pred, average='micro', multi_class='ovr', labels=cohorts_test)

0.5063227718037842

In [27]:
np.nanmean(np.array(lst_of_auc), axis=0)

0.4980368353047825

In [28]:
roc_auc_score(y_test, y_pred, average='macro', multi_class='ovr', labels=cohorts_test)

0.5063227718037842

In [29]:
y_test[cohorts_test == tasks[0]]

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,

In [30]:
y_pred[cohorts_test == tasks[0]] > 0.5

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [